In [3]:
from orion_recommend.datasets import DatasetMap, synthetic, utils
from orion.sources import S3Source
from orion.sources.io import read_csv, write_csv
from orion.sources import RedshiftSource
import pandas as pd
import numpy as np
from scipy import sparse

In [4]:
#source = RedshiftSource(query='SELECT * FROM publish.inventory_lookup WHERE productid IS NULL')
#df_stock = source.read_csv()

with S3Source(key="masters/uploads/customers/1560425511130_Peak_customers.csv", bucket="kilimanjaro-prod-datalake") as s3:
    df_cust = read_csv(s3)

df_trans = read_csv(S3Source(key="masters/uploads/transactions/1560426066385_Peak_transactions.csv", bucket="kilimanjaro-prod-datalake"))
df_prod = read_csv(S3Source(key="masters/uploads/product/1560425499995_Peak_product.csv", bucket="kilimanjaro-prod-datalake"))


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# 30pc of transactions have keys that are not in df_cust
pd.Series(df_trans.customerkey.unique()).isin(df_cust.customerkey).sum()/len(df_trans.customerkey.unique())
# 45pc of item numbers not in df_prod
pd.Series(df_trans.itemnumber.unique()).isin(df_prod.itemnumber).sum()/len(df_trans.itemnumber.unique())

0.5487752814984791

In [6]:
# Fix item number 
df_prod.itemnumber = pd.to_numeric(df_prod.itemnumber, errors='coerce') 

In [7]:
# Drop nas
df_prod.dropna(subset=["itemnumber"], inplace=True)

In [8]:
# Remove keys not in customer and product tables
df_trans = df_trans.iloc[df_trans.customerkey[df_trans.customerkey.isin(df_cust.customerkey)].index,:]
df_trans.reset_index(inplace=True, drop=True)
df_trans = df_trans.iloc[df_trans.itemnumber[df_trans.itemnumber.isin(df_prod.itemnumber)].index,:]

# Reset index before changing keys to integers
df_trans.reset_index(inplace=True, drop=True)
df_prod.reset_index(inplace=True, drop=True)


In [9]:
df_prod.dropna(inplace=True)
df_prod.itemnumber.unique().shape

(71993,)

In [10]:
print(df_cust.customerkey.unique().shape)
df_cust.dropna(inplace=True)
print(df_cust.customerkey.unique().shape)

(2434521,)
(1701289,)


In [11]:
# Drop redundant cols. Fabric all unknown, item colour less info than colourvalue
df_prod.drop(["itemstylecode", "itemquarter", "itemcolour", "fabric"], axis=1, inplace=True)

In [12]:
# Drop columns
df_trans.drop(["salestransactionkey", "salesordernumber","discountpercent", "grosssales",
              'orderdate', 'ordertime', 'shippingdate','grossprofit'], axis=1, inplace=True)

In [13]:
df = df_trans.merge(df_cust, on='customerkey', how = "outer")
df = df.merge(df_prod, on = "itemnumber")

In [14]:
# Rename and drop columns, nas
df.rename(columns={"customerkey":"user_id","itemnumber":"item_id","shipcountry":"country", "brandcode":"brand_id",
                  "colourvalue":"colour", }, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [15]:
df_trans.rename(columns={"customerkey":"user_id","itemnumber":"item_id"}, inplace=True)
df_cust.rename(columns={"customerkey":"user_id"}, inplace=True)
df_prod.rename(columns={"itemnumber":"item_id"}, inplace=True)

In [16]:
cats = df.select_dtypes('object').columns
keys = df[cats].apply(lambda x: x.factorize()[1])
df[cats] = df[cats].apply(lambda x: x.factorize()[0])

In [17]:
user_num = len(df.user_id.unique())
item_num = len(df.item_id.unique())

In [18]:
df.head()

,user_id,country,postcode,item_id,pricetype,unitssold,loyaltyaccount,gender,brand_id,category,colour,divisioncode,itemcategorycode,itemfamilycode,itemseason,productgroup
0,0,0,0,90956.0,0,1.0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,90956.0,0,1.0,1,1,0,0,0,0,0,0,0,0
2,2,0,2,90956.0,0,1.0,1,1,0,0,0,0,0,0,0,0
3,2,0,2,90956.0,0,1.0,1,1,0,0,0,0,0,0,0,0
4,2,0,2,90956.0,0,1.0,1,1,0,0,0,0,0,0,0,0


In [19]:
#Renumber item ids
ints = [i for i in range(item_num)]
item_keys = [i for i in df.item_id.unique()]
item_key_dict = {i:j for i,j in zip( item_keys, ints)}

In [20]:
# Change keys to integers in dataframes
df.item_id = [item_key_dict[df.item_id[i]] for i in range(len(df))]


In [21]:
df.drop('unitssold', axis=1, inplace=True)

In [22]:
len(df.item_id.unique())

36087

In [23]:
df.user_id.unique().shape

(1528454,)

In [24]:
# Train-test split
train = df.sample(frac = 0.8)
ind = df.index.isin(train.index)
test = df[~ind]

In [25]:
np.save("fa_train_data",train.to_numpy())
np.save("fa_test_data",test.to_numpy())

In [26]:
USER_NUM = len(df.user_id.unique())
country_num = len(df.country.unique())
postcode_num = len(df.postcode.unique())	
item_id_num = len(df.item_id.unique())
pricetype_num = len(df.pricetype.unique())
loyaltyaccount_num = len(df.loyaltyaccount.unique())
gender_num = len(df.gender.unique())
brand_id_num = len(df.brand_id.unique())
category_num = len(df.category.unique())
colour_num = len(df.colour.unique())
divisioncode_num = len(df.divisioncode.unique())
itemcategorycode_num = len(df.itemcategorycode.unique())
itemfamilycode_num = len(df.itemfamilycode.unique())
itemseason_num = len(df.itemseason.unique())
productgroup_num = len(df.productgroup.unique())

In [27]:
print(USER_NUM,
country_num,
postcode_num,
item_id_num,
pricetype_num,
loyaltyaccount_num,
gender_num,
brand_id_num,
category_num,
colour_num,
divisioncode_num,
itemcategorycode_num,
itemfamilycode_num,
itemseason_num,
productgroup_num)

1528454 68 17772 36087 2 2 3 329 8 59 3 13 7 3 78


### Item Attribute matrix

In [24]:
features = [  pricetype_num, brand_id_num, category_num, colour_num, divisioncode_num, 
            itemcategorycode_num, itemfamilycode_num, itemseason_num, productgroup_num]
names = [ 'pricetype', 'brand_id',
         'category', 'colour', 'divisioncode', 'itemcategorycode',
         'itemfamilycode', 'itemseason', 'productgroup']
dic = dict(zip(names,features))

In [86]:
matrices = []
for i in dic.keys():
    mat = np.zeros((item_num, dic[i]), dtype = np.int)    
    pair = df.loc[:,["item_id", i]]
    M = np.unique(pair, axis=0)
    for m in M:
        mat[m[0], m[1]] = 1
    print(mat[1])
    matrices.append(mat)

[1 1]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0]
[0 1 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [87]:
fa_matrix = np.concatenate((matrices), axis=1)

In [88]:
np.save('fa_ia_matrix.npy', fa_matrix)

In [89]:
matrices[0].shape

(36087, 2)

### User- attribute matrix

In [25]:
matrices = []
for i in dic.keys():
    mat = np.zeros((USER_NUM, dic[i]), dtype = np.int)
    pair = df.loc[:,["user_id", i]]
    M = np.unique(pair, axis=0)
    for m in M:
        mat[m[0], m[1]] = 1
    print(mat[1])
    matrices.append(mat)

[1 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0]
[1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [26]:
fa_matrix = np.concatenate((matrices), axis=1)

In [1]:
fa_matrix.shape

NameError: name 'fa_matrix' is not defined

In [28]:
np.save('fa_ua_matrix.npy', fa_matrix)